# IUM projekt - Zniżki dla klientów

## Zadanie
- Zinterpretować: “Niektórzy  klienci  nie  mogą  zdecydować  się  na zakup oglądając  produkt  na  stronie – ale pewnie  gdybyśmy  wiedzieli,  które  to  są  osoby,  to odpowiednia zniżka  skłoniłaby ich do zakupu."  
- Analiza danych.  
- Doprecyzowanie, przeanalizowanie problemu i wdrozenie aplikacji bazującej na mikroserwisach.  

## Problem biznesowy
Stworzenie systemu, który umożliwi zaproponowanie klientom niezdecydowanym, zniżki takiej, która zachęci ich do zakupu, ale równocześnie bedzie jak najmniejsza.  
Istnieje kilka interpretacji zadania:  
1. Model ma mówić czy klient jest niezdecydowany
2. Model ma mówić czy klient jest niezdecydowany i jakoś optymalizować stawkę zniżki
3. Model ma mówić czy opłaca się zaproponować zniżkę klientowi
4. Model ma wyznaczać wartośc zniżki którą powinno się zaproponować każdemu klientówi na podstawie jego historii
5. Przewidywać, po ilu wyświetlonych produktach klient, będzie chciał kupić produkt.  

Zakładamy interpretację nr.4.

## Zadania modelowania
- zadanie grupowania ( grupujemy klientów, żeby wspomóc przewidywanie zniżki )
- zadanie regresji ( znalezienie takiej znizki, która spowoduje, że klient zdecyduje się na zakup, ale firma zarobi jak najwięcej )

## Założenia
* Będziemy na zbiorze testowym sprawdzać czy dobrze przewidzieliśmy, że klient dokona zakupu. To znaczy, że sprawdzamy historie klienta i jeżeli wyświetlał produkty a przy danej zniżce i dokonał zakupu, to oznacza, że nasz model również powinien to przewidzieć.
* Odpowiedna zniżka to taka, która zachęci klienta do zakupu. Przy czym musi ona przynieść jak najwiekszy zysk firmie. To znaczy, że znajdujemy pierwszą zniżkę dla której model przewidzi, że klient dokona zakupu.
* Zakładamy, że testów online nie będziemy mogli przeprowadzić, ograniczymy się wiec do estymacji jakości modelu.

## Kryteria sukcesu
- Stworzenie systemu, który przyniesie jak największy zysk firmie.  
Składa się na to zmotywowanie do zakupu jak największej ilości klientów, ale nie kosztem straty wynikającej z za dużej zniżki.  
- Chcemy, żeby nasz model miał jak największy stosunek *liczby zaoferowanych zniżek* / *sumy oferowanych zniżek*

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd

## Wykonujemy czyszczenie danych
Procedura opisana w pliku clean_data.ipynb

In [ ]:
products_clean = pd.read_json(path_or_buf='data/products_clean.jsonl', lines=True)
products_clean.info()

In [ ]:
sessions_clean = pd.read_json(path_or_buf='data/sessions_clean.jsonl', lines=True)
sessions_clean.info()

## Potencjalne cechy modelu klasyfikacji
Na podstawie wyżej wypisanych danych można wnioskować, że da się z nich utworzyć cechy: 
- ile razy dany klient przegladal ten produkt
- ile razy dany klient przegladal dana kategorie
- ile razy dany klient w tej sesji przegladal produkty z tej kategorii,
- ile razy znizka zachecila go do zakupu
- ile razy kupowal cos z tej kategorii
  
W tym celu wymagany bedzie dostęp do historii eventów klienta oraz dostęp do jego najnowszych akcji.  
Nasza aplikacja powinna przetworzyć i przechowywać niektóre cechy klienta, żeby poźniej sieć neuronowa mogła na podstawie nich wnioskować, jak bardzo niezdecydowany jest klient.
Kiedy nasza aplikacja dostanie informację o nowej akcji klienta, powinna już wiedzieć jak zachowywał się w przeszłości i uwzględnić to w przewidywaniu.

## Labelowanie danych
Do uczenia naszej sieci neuronowej będziemy musieli ocenić w jakiś sposób którzy klienci byli niezdecydowani.  
Najbardziej wymiernym sposobem byłoby dodanie do strony ankiety, w której dany klient mógłby podać jak bardzo niezdecydowany jest co do zakupu. (np. w skali od 1-6)  
Dzięki temu nasza się uczyłaby się rozpoznawać to jak się czują faktyczni klienci i po jakimś czasie nie musielibyśmy pytać ich oto w ankiecie.  
  
  
Drugą opcją jest automatyczne labelowanie dotychczasowych danych. W tym celu należało będzie utworzyć funkcję labelującą czy dany klient, na podstawie dotychczasowych danych, jest niezdecydowany czy nie. Będzie to wymagało niestety pewnej dozy heurystyki, więc nasze nasza sieć będzie się uczyć pewnego rodzaju schematu. Możnaby próbować to zredukować wprowadzając uczenie online, które będzie się douczało po wdrożeniu sieci neuronowej. 
